In [144]:
import numpy as np
import pandas as pd
import deeppavlov
import json
import codecs
from copy import deepcopy

In [145]:
with codecs.open("../data/train.jsonl", "r", "utf-8") as stream:
    data = stream.readlines()
    data = list(map(json.loads, data))
    print(data[0])

{'idx': 0, 'passage': {'text': '(1) Но люди не могут существовать без природы, поэтому в парке стояли железобетонные скамейки — деревянные моментально ломали. (2) В парке бегали ребятишки, водилась шпана, которая развлекалась игрой в карты, пьянкой, драками, «иногда насмерть». (3) «Имали они тут и девок...» (4) Верховодил шпаной Артемка-мыло, с вспененной белой головой. (5) Людочка сколько ни пыталась усмирить лохмотья на буйной голове Артемки, ничего у неё не получалось. (6) Его «кудри, издали напоминавшие мыльную пену, изблизя оказались что липкие рожки из вокзальной столовой — сварили их, бросили комком в пустую тарелку, так они, слипшиеся, неподъёмно и лежали. (7) Да и не ради причёски приходил парень к Людочке. (8) Как только её руки становились занятыми ножницами и расчёской, Артемка начинал хватать её за разные места. (9) Людочка сначала увёртывалась от хватких рук Артемки, а когда не помогло, стукнула его машинкой по голове и пробила до крови, пришлось лить йод на голову «ухажо

In [146]:
import re

tokenize_sentence(data[308]['passage']['text'])

['Набегали тучки.',
 'Небо хмурилось.',
 'Серенькие вихри завивались и бежали по дороге, маня и дразня кого-то.',
 'Лиза остановила Сеппа: – Послушайте, Пауль, подойдите ко мне на минутку.',
 'Пауль отошёл на боковую тропинку.',
 'Он шёл рядом с Лизою.',
 'Походка его была решительная и твёрдая, и глаза смело глядели вперёд.',
 'Казалось, что в душе его ритмично бились торжественные звуки воинственной музыки.',
 'Лиза смотрела на него влюблёнными глазами.',
 'Он сказал: – Ничего не бойтесь, Лиза.',
 'Пока мы живы, мы немцев далеко не пустим.',
 'А кто войдёт в Россию, тот не обрадуется нашему приёму.',
 'Чем больше их войдёт, тем меньше их вернётся в Германию.',
 'Вдруг Лиза очень покраснела и сказала: – Пауль, в эти дни я вас полюбила.',
 'Я поеду за вами.',
 'Меня возьмут в сёстры милосердия.',
 'При первой возможности мы повенчаемся.',
 'Пауль вспыхнул.',
 'Он наклонился, поцеловал Лизину руку и повторял: – Милая, милая!',
 'И когда он опять посмотрел в её лицо, его ясные глаза были

In [147]:
data[308]['passage']['text']

'(1) (1)Набегали тучки. (2)Небо хмурилось. (3)Серенькие вихри завивались и бежали по дороге, маня и дразня кого-то. (4)Лиза остановила Сеппа: – Послушайте, Пауль, подойдите ко мне на минутку. (5)Пауль отошёл на боковую тропинку. (6)Он шёл рядом с Лизою. (7)Походка его была решительная и твёрдая, и глаза смело глядели вперёд. (8)Казалось, что в душе его ритмично бились торжественные звуки воинственной музыки. (9)Лиза смотрела на него влюблёнными глазами. (10)Он сказал: – Ничего не бойтесь, Лиза. (11)Пока мы живы, мы немцев далеко не пустим. (12)А кто войдёт в Россию, тот не обрадуется нашему приёму. (13)Чем больше их войдёт, тем меньше их вернётся в Германию. (514)Вдруг Лиза очень покраснела и сказала: – Пауль, в эти дни я вас полюбила. (15)Я поеду за вами. (16)Меня возьмут в сёстры милосердия. (17)При первой возможности мы повенчаемся. (18)Пауль вспыхнул. (19)Он наклонился, поцеловал Лизину руку и повторял: – Милая, милая! (20)И когда он опять посмотрел в её лицо, его ясные глаза были 

In [148]:
# for i in range(len(data)):
#     data[i]['passage']['text'] = ... #TODO

In [149]:
from deeppavlov.models.tokenizers.ru_tokenizer import RussianTokenizer 
from deeppavlov.models.embedders.fasttext_embedder import FasttextEmbedder
from deeppavlov.models.embedders.tfidf_weighted_embedder import TfidfWeightedEmbedder

In [150]:
def tokenize_sentence(text):
    sentences = re.split("\(\d+\)",text)
    sentences = [_.strip() for _ in sentences if _.strip() != '']
    return sentences

In [151]:
model_path = '../data/ft_native_300_ru_wiki_lenta_lemmatize.bin'

tokenizer = RussianTokenizer(lemmas=True, alphas_only=False)

In [152]:
for i in range(len(data)):
    if i % 300 == 0:
        print(i)
    text = data[i]['passage']['text']
    sentences = tokenize_sentence(text)
    data[i]['passage']['text'] = [tokenizer([_])[0] for _ in sentences]
    for j in range(len(data[i]['passage']['questions'])):
        question = data[i]['passage']['questions'][j]['question']
        data[i]['passage']['questions'][j]['question'] = tokenizer([question])[0]
        for k in range(len(data[i]['passage']['questions'][j]['answers'])):
            answer = data[i]['passage']['questions'][j]['answers'][k]['text']
            data[i]['passage']['questions'][j]['answers'][k]['text'] = tokenizer([answer])[0]

0
300


In [140]:
model = FasttextEmbedder(model_path)

2020-11-22 17:27:41.886 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 53: [loading fastText embeddings from `/mnt/data/ft_native_300_ru_wiki_lenta_lemmatize.bin`]


In [153]:
for i in range(len(data)):
    if i % 100 == 0:
        print(i)
    sentences = data[i]['passage']['text']
    data[i]['passage']['text'] = [np.max(model([_])[0], axis=0) for _ in sentences]
    for j in range(len(data[i]['passage']['questions'])):
        question = data[i]['passage']['questions'][j]['question']
        data[i]['passage']['questions'][j]['question'] = np.max(model([question])[0], axis=0)
        for k in range(len(data[i]['passage']['questions'][j]['answers'])):
            answer = data[i]['passage']['questions'][j]['answers'][k]['text']
            data[i]['passage']['questions'][j]['answers'][k]['text'] = np.max(model([answer])[0], axis=0)

0
100
200
300
400


In [154]:
for i in range(len(data)):
    if i % 100 == 0:
        print(i)
    sentences = data[i]['passage']['text']
    for j in range(len(data[i]['passage']['questions'])):
        question = data[i]['passage']['questions'][j]['question']
        dists = []
        for sentence in sentences:
            dists.append(np.linalg.norm(sentence - question))
        idx = np.argmin(dists)
        data[i]['passage']['questions'][j]['text'] = sentences[idx]

0
100
200
300
400


In [155]:
X = []
y = []
for i in range(len(data)):
#     text = data[i]['passage']['text']
    for j in range(len(data[i]['passage']['questions'])):
        question = data[i]['passage']['questions'][j]['question']
        text = data[i]['passage']['questions'][j]['text']
        for k in range(len(data[i]['passage']['questions'][j]['answers'])):
            answer = data[i]['passage']['questions'][j]['answers'][k]['text']
            label = data[i]['passage']['questions'][j]['answers'][k]['label']
            row = np.concatenate([text, question, answer])
            X.append(row)
            y.append(label)

In [156]:
with codecs.open("../data/val.jsonl", "r", "utf-8") as stream:
    val = stream.readlines()
    val = list(map(json.loads, val))
    
X_val = []
y_val = []
for i in range(len(val)):
    text = val[i]['passage']['text']
    sentences = tokenize_sentence(text)
    sentences = [tokenizer([_])[0] for _ in sentences]
    sentences = [np.max(model([_])[0], axis=0) for _ in sentences]
    for j in range(len(val[i]['passage']['questions'])):
        question = val[i]['passage']['questions'][j]['question']
        question = tokenizer([question])[0]
        question = np.max(model([question])[0], axis=0)
        dists = []
        for sentence in sentences:
            dists.append(np.linalg.norm(sentence - question))
        idx = np.argmin(dists)
        text = sentences[idx]
        for k in range(len(val[i]['passage']['questions'][j]['answers'])):
            answer = val[i]['passage']['questions'][j]['answers'][k]['text']
            answer = tokenizer([answer])[0]
            answer = np.max(model([answer])[0], axis=0)
            row = np.concatenate([text, question, answer])
            X_val.append(row)
            label = val[i]['passage']['questions'][j]['answers'][k]['label']
            y_val.append(label)


In [157]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
net = CatBoostClassifier()

net.fit(X, y)

Learning rate set to 0.029715
0:	learn: 0.6910380	total: 106ms	remaining: 1m 45s
1:	learn: 0.6890744	total: 177ms	remaining: 1m 28s
2:	learn: 0.6870124	total: 272ms	remaining: 1m 30s
3:	learn: 0.6851273	total: 346ms	remaining: 1m 26s
4:	learn: 0.6832049	total: 418ms	remaining: 1m 23s
5:	learn: 0.6815885	total: 510ms	remaining: 1m 24s
6:	learn: 0.6797807	total: 588ms	remaining: 1m 23s
7:	learn: 0.6782394	total: 671ms	remaining: 1m 23s
8:	learn: 0.6765424	total: 747ms	remaining: 1m 22s
9:	learn: 0.6750872	total: 820ms	remaining: 1m 21s
10:	learn: 0.6734844	total: 891ms	remaining: 1m 20s
11:	learn: 0.6720874	total: 962ms	remaining: 1m 19s
12:	learn: 0.6706568	total: 1.04s	remaining: 1m 19s
13:	learn: 0.6692064	total: 1.11s	remaining: 1m 18s
14:	learn: 0.6678906	total: 1.19s	remaining: 1m 18s
15:	learn: 0.6667533	total: 1.27s	remaining: 1m 17s
16:	learn: 0.6654399	total: 1.34s	remaining: 1m 17s
17:	learn: 0.6642250	total: 1.42s	remaining: 1m 17s
18:	learn: 0.6630587	total: 1.5s	remaining: 

157:	learn: 0.6056503	total: 15.4s	remaining: 1m 22s
158:	learn: 0.6054819	total: 15.5s	remaining: 1m 22s
159:	learn: 0.6052868	total: 15.6s	remaining: 1m 22s
160:	learn: 0.6050500	total: 15.7s	remaining: 1m 22s
161:	learn: 0.6048627	total: 15.8s	remaining: 1m 21s
162:	learn: 0.6046554	total: 16s	remaining: 1m 21s
163:	learn: 0.6043782	total: 16s	remaining: 1m 21s
164:	learn: 0.6041013	total: 16.2s	remaining: 1m 21s
165:	learn: 0.6038399	total: 16.3s	remaining: 1m 21s
166:	learn: 0.6036321	total: 16.4s	remaining: 1m 21s
167:	learn: 0.6033310	total: 16.5s	remaining: 1m 21s
168:	learn: 0.6030741	total: 16.6s	remaining: 1m 21s
169:	learn: 0.6028165	total: 16.7s	remaining: 1m 21s
170:	learn: 0.6025610	total: 16.8s	remaining: 1m 21s
171:	learn: 0.6022767	total: 16.9s	remaining: 1m 21s
172:	learn: 0.6020830	total: 17s	remaining: 1m 21s
173:	learn: 0.6018545	total: 17.1s	remaining: 1m 21s
174:	learn: 0.6016301	total: 17.3s	remaining: 1m 21s
175:	learn: 0.6013644	total: 17.4s	remaining: 1m 21s

314:	learn: 0.5676685	total: 32.5s	remaining: 1m 10s
315:	learn: 0.5674118	total: 32.6s	remaining: 1m 10s
316:	learn: 0.5671421	total: 32.7s	remaining: 1m 10s
317:	learn: 0.5669727	total: 32.8s	remaining: 1m 10s
318:	learn: 0.5667822	total: 32.9s	remaining: 1m 10s
319:	learn: 0.5664767	total: 33s	remaining: 1m 10s
320:	learn: 0.5661888	total: 33.1s	remaining: 1m 10s
321:	learn: 0.5659014	total: 33.2s	remaining: 1m 9s
322:	learn: 0.5656825	total: 33.3s	remaining: 1m 9s
323:	learn: 0.5655349	total: 33.4s	remaining: 1m 9s
324:	learn: 0.5652958	total: 33.5s	remaining: 1m 9s
325:	learn: 0.5650016	total: 33.6s	remaining: 1m 9s
326:	learn: 0.5646878	total: 33.8s	remaining: 1m 9s
327:	learn: 0.5644857	total: 33.8s	remaining: 1m 9s
328:	learn: 0.5642472	total: 34s	remaining: 1m 9s
329:	learn: 0.5639464	total: 34.1s	remaining: 1m 9s
330:	learn: 0.5636913	total: 34.2s	remaining: 1m 9s
331:	learn: 0.5634640	total: 34.3s	remaining: 1m 8s
332:	learn: 0.5633053	total: 34.4s	remaining: 1m 8s
333:	lear

474:	learn: 0.5199344	total: 49.6s	remaining: 54.8s
475:	learn: 0.5195327	total: 49.7s	remaining: 54.7s
476:	learn: 0.5192089	total: 49.9s	remaining: 54.7s
477:	learn: 0.5188328	total: 50s	remaining: 54.6s
478:	learn: 0.5186161	total: 50.1s	remaining: 54.5s
479:	learn: 0.5183561	total: 50.2s	remaining: 54.4s
480:	learn: 0.5180410	total: 50.3s	remaining: 54.3s
481:	learn: 0.5177311	total: 50.4s	remaining: 54.2s
482:	learn: 0.5174561	total: 50.5s	remaining: 54.1s
483:	learn: 0.5172192	total: 50.6s	remaining: 54s
484:	learn: 0.5169609	total: 50.7s	remaining: 53.9s
485:	learn: 0.5166598	total: 50.8s	remaining: 53.8s
486:	learn: 0.5164495	total: 51s	remaining: 53.7s
487:	learn: 0.5161420	total: 51.1s	remaining: 53.6s
488:	learn: 0.5159011	total: 51.2s	remaining: 53.5s
489:	learn: 0.5156082	total: 51.3s	remaining: 53.4s
490:	learn: 0.5153152	total: 51.4s	remaining: 53.3s
491:	learn: 0.5149311	total: 51.5s	remaining: 53.2s
492:	learn: 0.5146043	total: 51.6s	remaining: 53.1s
493:	learn: 0.5142

634:	learn: 0.4763734	total: 1m 7s	remaining: 38.6s
635:	learn: 0.4761183	total: 1m 7s	remaining: 38.5s
636:	learn: 0.4758968	total: 1m 7s	remaining: 38.4s
637:	learn: 0.4756079	total: 1m 7s	remaining: 38.3s
638:	learn: 0.4753734	total: 1m 7s	remaining: 38.2s
639:	learn: 0.4750589	total: 1m 7s	remaining: 38.1s
640:	learn: 0.4747798	total: 1m 7s	remaining: 38s
641:	learn: 0.4745340	total: 1m 7s	remaining: 37.9s
642:	learn: 0.4742688	total: 1m 8s	remaining: 37.8s
643:	learn: 0.4740638	total: 1m 8s	remaining: 37.7s
644:	learn: 0.4737515	total: 1m 8s	remaining: 37.6s
645:	learn: 0.4734789	total: 1m 8s	remaining: 37.5s
646:	learn: 0.4732587	total: 1m 8s	remaining: 37.4s
647:	learn: 0.4729645	total: 1m 8s	remaining: 37.3s
648:	learn: 0.4727867	total: 1m 8s	remaining: 37.2s
649:	learn: 0.4725756	total: 1m 8s	remaining: 37.1s
650:	learn: 0.4722859	total: 1m 8s	remaining: 37s
651:	learn: 0.4720670	total: 1m 9s	remaining: 36.9s
652:	learn: 0.4717996	total: 1m 9s	remaining: 36.8s
653:	learn: 0.47

790:	learn: 0.4388259	total: 1m 25s	remaining: 22.7s
791:	learn: 0.4385864	total: 1m 25s	remaining: 22.6s
792:	learn: 0.4383170	total: 1m 26s	remaining: 22.5s
793:	learn: 0.4381277	total: 1m 26s	remaining: 22.4s
794:	learn: 0.4378666	total: 1m 26s	remaining: 22.3s
795:	learn: 0.4376496	total: 1m 26s	remaining: 22.2s
796:	learn: 0.4374737	total: 1m 26s	remaining: 22.1s
797:	learn: 0.4371914	total: 1m 26s	remaining: 22s
798:	learn: 0.4370382	total: 1m 26s	remaining: 21.9s
799:	learn: 0.4367641	total: 1m 26s	remaining: 21.7s
800:	learn: 0.4365874	total: 1m 27s	remaining: 21.6s
801:	learn: 0.4363228	total: 1m 27s	remaining: 21.5s
802:	learn: 0.4361200	total: 1m 27s	remaining: 21.4s
803:	learn: 0.4358797	total: 1m 27s	remaining: 21.3s
804:	learn: 0.4356257	total: 1m 27s	remaining: 21.2s
805:	learn: 0.4353534	total: 1m 27s	remaining: 21.1s
806:	learn: 0.4351260	total: 1m 27s	remaining: 21s
807:	learn: 0.4348198	total: 1m 27s	remaining: 20.9s
808:	learn: 0.4345970	total: 1m 28s	remaining: 20.

946:	learn: 0.4055926	total: 1m 44s	remaining: 5.83s
947:	learn: 0.4053630	total: 1m 44s	remaining: 5.72s
948:	learn: 0.4051097	total: 1m 44s	remaining: 5.61s
949:	learn: 0.4049354	total: 1m 44s	remaining: 5.5s
950:	learn: 0.4046618	total: 1m 44s	remaining: 5.39s
951:	learn: 0.4044599	total: 1m 44s	remaining: 5.28s
952:	learn: 0.4042247	total: 1m 44s	remaining: 5.17s
953:	learn: 0.4039417	total: 1m 44s	remaining: 5.06s
954:	learn: 0.4037394	total: 1m 45s	remaining: 4.95s
955:	learn: 0.4035298	total: 1m 45s	remaining: 4.84s
956:	learn: 0.4033689	total: 1m 45s	remaining: 4.73s
957:	learn: 0.4032166	total: 1m 45s	remaining: 4.62s
958:	learn: 0.4030705	total: 1m 45s	remaining: 4.51s
959:	learn: 0.4028417	total: 1m 45s	remaining: 4.4s
960:	learn: 0.4026862	total: 1m 45s	remaining: 4.29s
961:	learn: 0.4024795	total: 1m 45s	remaining: 4.18s
962:	learn: 0.4022889	total: 1m 45s	remaining: 4.07s
963:	learn: 0.4020835	total: 1m 46s	remaining: 3.96s
964:	learn: 0.4019062	total: 1m 46s	remaining: 3

In [158]:
predicted = net.predict(X_val)

In [159]:
from sklearn.metrics import accuracy_score

accuracy_score(y_val, predicted)

0.6317673378076063

In [129]:
with codecs.open("../data/test.jsonl", "r", "utf-8") as stream:
    test = stream.readlines()
    test = list(map(json.loads, test))
for i in range(len(test)):
    text = test[i]['passage']['text']
    sentences = tokenize_sentence(text)
    sentences = [tokenizer([_])[0] for _ in sentences]
    sentences = [np.mean(model([_])[0], axis=0) for _ in sentences]
    for j in range(len(test[i]['passage']['questions'])):
        question = test[i]['passage']['questions'][j]['question']
        question = tokenizer([question])[0]
        question = np.mean(model([question])[0], axis=0)
        dists = []
        for sentence in sentences:
            dists.append(np.linalg.norm(sentence - question))
        idx = np.argmin(dists)
        text = sentences[idx]
        for k in range(len(test[i]['passage']['questions'][j]['answers'])):
            answer = test[i]['passage']['questions'][j]['answers'][k]['text']
            answer = tokenizer([answer])[0]
            answer = np.mean(model([answer])[0], axis=0)
            row = np.concatenate([text, question, answer])
            test[i]['passage']['questions'][j]['answers'][k]['label'] = int(net.predict([row])[0])

In [130]:
with open('../data/submit.jsonl', 'w') as stream:
    for i in range(len(test)):
        try:
            stream.write(json.dumps(test[i], ensure_ascii=False) + '\n')
        except Exception as e:
            print(e)
#             print(test[i])
            print(i)

In [47]:
test[0]

{'idx': 0,
 'passage': {'text': '"(1) Издательство Viking Press совместно с компанией TradeMobile выпустят мобильное приложение, посвященное Анне Франк, передает The Daily Telegraph. (2) Программа будет включать в себя фрагменты из дневника Анны, озвученные британской актрисой Хеленой Бонэм Картер. (3) Помимо этого, в приложение войдут фотографии и видеозаписи, документы из архива Фонда Анны Франк, план здания в Амстердаме, где Анна с семьей скрывались от нацистов, и факсимильные копии страниц дневника. (4) Приложение, которое получит название Anne Frank App, выйдет 18 октября. (5) Интерфейс программы будет англоязычным. (6) На каких платформах будет доступно Anne Frank App, не уточняется. Анна Франк родилась в Германии в 1929 году. (7) Когда в стране начались гонения на евреев, Анна с семьей перебрались в Нидерланды. (8) С 1942 года члены семьи Франк и еще несколько человек скрывались от нацистов в потайных комнатах дома в Амстердаме, который занимала компания отца Анны. (9) В 1944 го